# Leje PDF/Image Scraper
#### Made By- Uday S.
#### For- Raphael G.

## I. Importing Libraries

In [1]:
#importing drivers
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, WebDriverException
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import time
#importing beautiful soup
from bs4 import BeautifulSoup

#importing the necessary libraries
import pandas as pd, numpy as np
#importing json package to read the data
import json

#importing os module
import os
from os.path import basename
#importing time
import time
#importing pathlib
import pathlib
#importing string
import string

## II. Defining Variables

In [2]:
#defining a function to find the position of a substring's nth instance in a string
def find_nth(haystack, needle, n):
    start = haystack.find(needle)
    while start >= 0 and n > 1:
        start = haystack.find(needle, start+len(needle))
        n -= 1
    return start

In [3]:
base_url= "https://www.leje.com.br"

## III. Scraping Page Links

In [4]:
# Using Selenium to define driver
driver = webdriver.Chrome(ChromeDriverManager().install())
#Defining base url
url= "https://www.leje.com.br/"
#Opening url
driver.get(url)
#Defining an empty variable
p_db=""
#getting the required element containing page links by class_name
p_element = driver.find_elements_by_class_name('auction-card-box ')

In [5]:
#Creating an empty DataFrame
url_df= pd.DataFrame()
#Creating an empty "url" column with the required rows
url_df["url"]= [None] * len(p_element)

In [6]:
#Using for loop to get the links of all the required pages
for i in range(len(p_element)):
    #getting the string having the link using get_attribute(), and assigning it to the column
    url_df["url"][i]=str(p_element[i].get_attribute("onclick"))
    #Cleaning the string to get the exact link for the page using find_nth function defined above
    url_df["url"][i]=url+url_df["url"][i][find_nth(url_df["url"][i],"index",1):find_nth(url_df["url"][i],"'",2)]

## IV. Defining Columns, Scraping Listing Names & Defining Directory

In [7]:
#Defining the Name column
url_df["Name"]= ""
#Defining the Name xpath to scrape data from
Name = '//*[@id="auction-title-text"]/p'
#Defining Drivers
driver = webdriver.Chrome(ChromeDriverManager().install())
#Using for loop to iterate through the pages
for i in range(len(p_element)):
    #Opening the url
    driver.get(url_df["url"][i])
    #Using Xpath to get the required element with listing name
    name_element= driver.find_element_by_xpath(Name)
    #Using .text to get the listing name and assigning to the url_df["Name"] column
    url_df["Name"][i]=name_element.text

In [8]:
#Defining Parent Directory
parent_dir = os.getcwd()
# Directory
directory = "Leje"
# Directory paths
path = os.path.join(parent_dir, directory)
# Create the directory
if not os.path.exists(path):
    os.mkdir(path)
else:
    pass

## V. Scraping PDFs

In [9]:
for n in range(len(url_df)):
    # URL from which pdfs to be downloaded
    url = url_df["url"][n]

    # Requests URL and get response object
    response = requests.get(url)

    # Parse text obtained
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all hyperlinks present on webpage
    links = soup.find_all('a')

    i = 0
    
    
    d1= url_df["Name"][n].replace("/"," ").replace('\ '," ")
    # Directory paths
    path1= os.path.join(path, d1)

    # Create the directory
    if not os.path.exists(path1):
        os.mkdir(path1)
    else:
        pass
    # From all links check for pdf link and
    # if present download file
    for link in links:
        if ('.pdf' in link.text):
            i += 1
            print("Downloading file: ", n,": ",i)

            # Get response object for link
            response = requests.get(base_url+link.get('href'))
            # Write content in pdf file
            if not os.path.exists(directory+"/"+d1+"/"+link.text.replace("\n","").replace("\t","")):
                                                    pdf = open(directory+"/"+d1+"/"+link.text.replace("\n","").replace("\t",""), 'wb')
                                                    pdf.write(response.content)
                                                    pdf.close()
                                                    print("File ", n,": ",i, " downloaded")
            else:
                                                   pass

## VI. Scraping Images

In [11]:
for n in range(len(url_df)):
    # URL from which pngs to be downloaded
    url = url_df["url"][n]

    # Requests URL and get response object
    response = requests.get(url)

    # Parse text obtained
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find all hyperlinks present on webpage
    links = soup.find_all('img')

    i = 0


    d1= url_df["Name"][n].replace("/"," ").replace('\ '," ")
    # Directory paths
    path1= os.path.join(path, d1)

    # Create the directory
    if not os.path.exists(path1):
        os.mkdir(path1)
    else:
        pass
    # From all links check for jpeg link and
    # if present download file
    for link in links:
        if "imagem" in str(link):
            try:
                i += 1

                print("Downloading file: ", n,": ",i)
                # Get response object for link
                response = requests.get(base_url+link.get("src"))
                # Write content in png file
                png = open(directory+"/"+d1+"/"+str(i)+".png", 'wb')
                png.write(response.content)
                png.close()
                print("File ", n,": ",i, " downloaded")
            except:
                pass